# OpenAI Function Calling

![](images/function_calling.png)

In [1]:
import os
import openai

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

How do we pass this information to the language model? \
OpenAI has exposed a new parameter called **functions**, which you can pass a list of function definitions.

In [3]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

- We are gonna pass these functions directly to the language model, the **description** parameters are really important.
- Language model will use these descriptions to determine whether to use this function.
- Any informantion you want the language model to have in order to determine whether to call a function or how to call a function should be in the **description.**

Now that we have these functions defined, let's use them in a call to the language model.
- First, we are gonna create a list of messages to pass to the language model.

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [5]:
import openai

In [6]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [7]:
print(response)

{
  "id": "chatcmpl-8HYNJJeR9dV77XeEJbamQSeG8yS7x",
  "object": "chat.completion",
  "created": 1699194405,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  }
}


Let's take a closer look at the response message.

In [8]:
response_message = response["choices"][0]["message"]

In [9]:
response_message

<OpenAIObject at 0x1a47f915760> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  }
}

In [10]:
response_message["content"]

In [11]:
response_message["function_call"]

<OpenAIObject at 0x1a47f9158a0> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
}

In [12]:
json.loads(response_message["function_call"]["arguments"])

{'location': 'Boston, MA'}

- The **arguments** parameter in **function_call** is a json dictionary itself, so we can use json.loads to load this into a python dictionary.
- The arguments that it passed back can be directly passed into the **get_current_weather** function.

In [13]:
args = json.loads(response_message["function_call"]["arguments"])

In [14]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

What happens if the message that you pass in isn't related to the function at all?\
Let's try it out, and see what happens then.

In [15]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [16]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [17]:
print(response)

{
  "id": "chatcmpl-8HYNMThVRuNJWW6EsOPfkyznSzyM8",
  "object": "chat.completion",
  "created": 1699194408,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  }
}


- We can see that the message thats returned has content and it doesn't have **function_call** parameter.
- The model is determining whether to use a function or not.
- There are additional parameters that we can pass in to force the model to use or not to use a function.

Let's take a look at those.

In [18]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto", # By default, it's set to auto, which means that the language model chooses.
)
print(response)

{
  "id": "chatcmpl-8HYNS8uIwbmiCcvJY9Mf04lh0phbb",
  "object": "chat.completion",
  "created": 1699194414,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  }
}


"Because we used 'auto,' the language model recognized that it didn't need to call the function, so it responded with role and content only."

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none", # This forces the language model not to use any of the functions provided.
)
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}, # Forcing it to call a function.
)
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

In [ ]:
messages.append(response["choices"][0]["message"])

In [ ]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

In [ ]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)